In [4]:
import json
import networkx as nx
from graphviz import Digraph
from collections import namedtuple, defaultdict

with open('NFA.json') as _:
    NFA = json.load(_)
Graph = nx.Graph()
startingState = NFA['startingState']

In [5]:
def add_epsilon_moves(queue: list) -> list:
    visited = []

    while queue:
        state = queue.pop(0) 
        visited.append(state)
        if '$' in NFA[state]:
            epsilon_value = NFA[state]['$']
            if isinstance(epsilon_value, list):
                for item in epsilon_value:
                    if item not in visited and item not in queue:
                        queue.append(item)
            else:
                if epsilon_value not in visited and epsilon_value not in queue:
                    queue.append(epsilon_value)
    return visited

def get_possible_inputs(states: set, input_list=None) -> set:
    inputs = set()
    for state in states:
        keys = list(input_list[state].keys()) if input_list is not None else list(NFA[state].keys())
        for key in keys[1:]:
            if key != '$' and key not in inputs:
                inputs.add(key)
    return inputs

def add_inputs(states: set, inputs: set) -> dict :
    state_inputs = {}
    for input in inputs:
        ns = set()
        for state in states:
            if(input in NFA[state]):
                ns.add(NFA[state][input])
        ns.update(add_epsilon_moves(list(ns)))
        state_inputs[input] = ns
    return state_inputs

def checkTerminating(states: set, input_list=None) -> bool:
    for state in states:
        state_dict = input_list[state] if input_list is not None else NFA[state]
        if state_dict['isTerminatingState']:
            return True
    return False

def sort_dict(d: dict):
    term_state = d.pop('isTerminatingState')
    sorted_dict = dict(sorted(d.items()))
    sorted_dict = {"isTerminatingState": term_state, **sorted_dict}
    return sorted_dict

In [6]:
states = set(); states_dict = {}
states.add(startingState)
ct = 0; nname = f"NS{ct}"
states.update(add_epsilon_moves([startingState]))

states_dict['startingState'] = nname
states_dict[nname] = {'isTerminatingState': checkTerminating(states)}; 
states_dict[nname]['states'] = states; ct+=1
states_dict[nname]['inp'] = ', '.join(map(str, states))

kv = list(states_dict.keys())
i = 0
while i < len(kv):
    ns = kv[i]
    ep_states = set().clear()
    if ns == "startingState":
        i += 1
        continue
    ep_states = add_epsilon_moves(list(states_dict[ns].get('states')))
    inputs = get_possible_inputs(ep_states)
    outStates = add_inputs(ep_states, inputs)
    for input in inputs:
        for key in list(states_dict.keys())[1:]:
            if(states_dict[key].get('states') == outStates[input]):
                nname = key
                break
        else:   #Only enter if not breaks
            nname = f"NS{ct}"
            states_dict[nname] = {'isTerminatingState': checkTerminating(outStates[input])}
            states_dict[nname]['states'] = outStates[input]
            states_dict[nname]['inp'] = ', '.join(map(str, outStates[input]))
            ct+=1
        states_dict[ns][input] = nname
    kv = list(states_dict.keys())
    i += 1

TypeError: unhashable type: 'list'

#### Drawing The Graphs

In [ ]:
cleaned_dict = {}
cleaned_dict['startingState'] = states_dict['startingState']
for key in states_dict:
    if key != "startingState":
        new_value = {k: v for k, v in states_dict[key].items() if (k != 'states' and k != 'inp')}
        cleaned_dict[key] = sort_dict(new_value)

with open('cleaned_DFA.json', 'w') as _:
    json.dump(cleaned_dict, _, indent=4)

In [ ]:
graph = Digraph(graph_attr={'rankdir': 'LR'})
graph.node('', shape='none')
for key in cleaned_dict:
    if key != 'startingState':
        if cleaned_dict[key]["isTerminatingState"]:
            graph.node(name= key,label= states_dict[key]['inp'], shape='doublecircle')
        else:    
            graph.node(name=key,label = states_dict[key]['inp'], shape='circle')

for key in cleaned_dict:
    if key != 'startingState':
        for states in cleaned_dict[key]:
            if states != 'isTerminatingState':
                graph.edge(key,cleaned_dict[key][states],states)
graph.edge('', states_dict['startingState'])
graph.unflatten().render('./dfa', view=True, format='png', cleanup=True)

graph = Digraph(graph_attr={'rankdir': 'LR'})
graph.node('', shape='none')
for key in cleaned_dict:
    if key != 'startingState':
        if cleaned_dict[key]["isTerminatingState"]:
            graph.node(name= key, shape='doublecircle')
        else:
            graph.node(name=key, shape='circle')

for key in cleaned_dict:
    if key != 'startingState':
        for states in cleaned_dict[key]:
            if states != 'isTerminatingState':
                graph.edge(key,cleaned_dict[key][states],states)
graph.edge('', states_dict['startingState'])
graph.unflatten().render('./cleaned_dfa', view=True, format='png', cleanup=True)


'cleaned_dfa.png'

### DFA Minimization

In [ ]:

Min_list = [[key for key, value in cleaned_dict.items() if key != "startingState" and not value['isTerminatingState']],
            [key for key, value in cleaned_dict.items() if key != "startingState" and value['isTerminatingState']]]

list_rows = Min_list
Row = namedtuple('Row', ['name','inputs', 'values'])

while 1:
    state_to_index = {}
    for i, sublist in enumerate(list_rows):
        for state in sublist:
            state_to_index[state] = i

    rows = []
    for lst in list_rows:
        for key in lst:
            dict_state = cleaned_dict[key]
            key_values = []; inputs = []; inputs_values = []

            for k, v in dict_state.items():
                if k != 'isTerminatingState':
                    inputs.append(k)
                    key_values.append(state_to_index.get(v))
                    # inputs_values.append(list(zip([k], [state_to_index.get(v)])))

            row = Row(key, inputs, (state_to_index.get(key), key_values))
            rows.append(row)

    dict_rows = defaultdict(list)
    for row in rows:
        dict_rows[(row.values[0], tuple(row.values[1]), tuple(row.inputs))].append(row.name)

    new_list_rows = list(map(list, dict_rows.values()))
    if new_list_rows == list_rows:
        break
    list_rows = new_list_rows

state_to_index = {}
for i, sublist in enumerate(list_rows):
    for state in sublist:
        state_to_index[state] = i


In [ ]:
fin_dict = {"startingState": f"S{state_to_index[cleaned_dict['startingState']]}"}
for fin_list in list_rows:
    fin_terminating = checkTerminating(set(fin_list), cleaned_dict)
    for item in fin_list:
        k = f"S{state_to_index[item]}"
        ns = {'isTerminatingState': fin_terminating}
        ns.update({it: f"S{state_to_index[cleaned_dict[item][it]]}" for it in cleaned_dict[item] if it != "isTerminatingState"})
        fin_dict[k]=ns
        break

with open('minimized_DFA.json', 'w') as _:
    json.dump(fin_dict, _, indent=4)

    
graph = Digraph(graph_attr={'rankdir': 'LR'})
graph.node('', shape='none')
for key in fin_dict:
    if key != 'startingState':
        if fin_dict[key]["isTerminatingState"]:
            graph.node(name= key, shape='doublecircle')
        else:
            graph.node(name=key, shape='circle')

for key in fin_dict:
    if key != 'startingState':
        for states in fin_dict[key]:
            if states != 'isTerminatingState':
                graph.edge(key,fin_dict[key][states],states)
graph.edge('', fin_dict['startingState'])
graph.unflatten().render('./minimized_dfa', view=True, format='png', cleanup=True)

'minimized_dfa.png'